In [1]:
import sys
from bs4 import BeautifulSoup
import urllib.request
import requests
import json
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fname = fm.FontProperties(fname='malgun.ttf').get_name()
import sqlite3
from IPython.display import Image
mpl.rc('font',family=fname)
import matplotlib.gridspec as gridspec
from jupyterthemes import jtplot
jtplot.style()
%matplotlib inline
import platform
platform.system()
# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

In [2]:
corona = pd.read_csv('Time.csv')
corona

,date,time,test,negative,confirmed,released,deceased
0,2020-01-20,16,1,0,1,0,0
1,2020-01-21,16,1,0,1,0,0
2,2020-01-22,16,4,3,1,0,0
3,2020-01-23,16,22,21,1,0,0
4,2020-01-24,16,27,25,2,0,0
...,...,...,...,...,...,...,...
158,2020-06-26,0,1232315,1200885,12602,11172,282
159,2020-06-27,0,1243780,1211261,12653,11317,282
160,2020-06-28,0,1251695,1219975,12715,11364,282
161,2020-06-29,0,1259954,1228698,12757,11429,282


### 현재 데이터는 누적 데이터 --> 일일 데이터로 바꿀 필요성이 있음

In [3]:
# 일일 확진자 수 구하기

daily_confirmed = []
for i in range(len(corona)-1):
    daily_confirmed.append(corona['confirmed'][i+1]-corona['confirmed'][i])
    
daily_confirmed.insert(0,1)

corona['daily_confirmed'] = daily_confirmed

In [4]:
# 일일 test

daily_test = []
for i in range(len(corona)-1):
    daily_test.append(corona['test'][i+1]-corona['test'][i])
    
daily_test.insert(0,1)

corona['daily_test'] = daily_test

In [5]:
# 일일 음성판정 수

daily_negative = []
for i in range(len(corona)-1):
    daily_negative.append(corona['negative'][i+1]-corona['negative'][i])
    
daily_negative.insert(0,0)

corona['daily_negative'] = daily_negative

In [6]:
# 일일 퇴원 환자 수

daily_released = []
for i in range(len(corona)-1):
    daily_released.append(corona['released'][i+1]-corona['released'][i])
    
daily_released.insert(0,0)

corona['daily_released'] = daily_released

In [7]:
# 일일 사망자

daily_confirmed = []
for i in range(len(corona)-1):
    daily_confirmed.append(corona['confirmed'][i+1]-corona['confirmed'][i])
    
daily_confirmed.insert(0,0)

corona['daily_confirmed'] = daily_confirmed

In [8]:
# 시간 데이터 형식 변환
corona['date'][0].replace('-','')

'20200120'

In [9]:
new_date = []
for i in range(len(corona)):
    new_date.append(corona['date'][i].replace('-',''))

In [10]:
corona['new_date'] = new_date

In [12]:
cols = corona.columns.tolist()
cols

['date',
 'time',
 'test',
 'negative',
 'confirmed',
 'released',
 'deceased',
 'daily_confirmed',
 'daily_test',
 'daily_negative',
 'daily_released',
 'new_date']

In [13]:
cols = cols[:-1]+cols[-1:]

In [14]:
corona = corona[cols]

In [15]:
corona

,date,time,test,negative,confirmed,released,deceased,daily_confirmed,daily_test,daily_negative,daily_released,new_date
0,2020-01-20,16,1,0,1,0,0,0,1,0,0,20200120
1,2020-01-21,16,1,0,1,0,0,0,0,0,0,20200121
2,2020-01-22,16,4,3,1,0,0,0,3,3,0,20200122
3,2020-01-23,16,22,21,1,0,0,0,18,18,0,20200123
4,2020-01-24,16,27,25,2,0,0,1,5,4,0,20200124
...,...,...,...,...,...,...,...,...,...,...,...,...
158,2020-06-26,0,1232315,1200885,12602,11172,282,39,11837,11870,198,20200626
159,2020-06-27,0,1243780,1211261,12653,11317,282,51,11465,10376,145,20200627
160,2020-06-28,0,1251695,1219975,12715,11364,282,62,7915,8714,47,20200628
161,2020-06-29,0,1259954,1228698,12757,11429,282,42,8259,8723,65,20200629


In [16]:
corona.drop(['date'],axis='columns',inplace=True)

In [21]:
corona

,time,test,negative,confirmed,released,deceased,daily_confirmed,daily_test,daily_negative,daily_released,new_date
0,16,1,0,1,0,0,0,1,0,0,20200120
1,16,1,0,1,0,0,0,0,0,0,20200121
2,16,4,3,1,0,0,0,3,3,0,20200122
3,16,22,21,1,0,0,0,18,18,0,20200123
4,16,27,25,2,0,0,1,5,4,0,20200124
...,...,...,...,...,...,...,...,...,...,...,...
158,0,1232315,1200885,12602,11172,282,39,11837,11870,198,20200626
159,0,1243780,1211261,12653,11317,282,51,11465,10376,145,20200627
160,0,1251695,1219975,12715,11364,282,62,7915,8714,47,20200628
161,0,1259954,1228698,12757,11429,282,42,8259,8723,65,20200629


In [26]:
# csv로 저장
corona.to_csv('.../코로나확진자.csv',index=False)

In [ ]:
# db에 추가하기 위해선 데이터 타입 변경 필요 (->int)

new_corona = new_corona.astype({"new_date":str,"daily_test":int,"daily_confirmed":int,"daily_negative":int,"daily_released":int})

### 데이터베이스에 추가

In [ ]:
dbname='KOSPI.db'
def addData_co(data):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql = 'insert into Corona_day \
        (CIDX,DATE,TEST,CONFIRMED,NEGATIVE,RELEASED) \
        values (?,?,?,?,?,?)'
        cur.execute(sql,data)
        conn.commit()
        cur.close()

In [ ]:
def selData_co(whr=' where CIDX=0'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql='select CIDX from Corona_day '+whr
        cur.execute(sql)
        rows=cur.fetchall()
        cur.close()
    return rows

In [ ]:
for i in range(len(new_corona)):
    cidx=new_corona['new_date'][i]+'1'
    data=[cidx,new_corona["new_date"][i],new_corona["daily_test"][i],new_corona["daily_confirmed"][i],
          new_corona["daily_negative"][i],new_corona["daily_released"][i]]
    preData=selData_co(' where CIDX='+cidx)
    if(preData):
        print("중복데이터 입니다.",preData)
    else:
        addData_co(data)